In [1]:
import wandb
import os
os.environ["WANDB_SILENT"] = "true"

run_name = "GOOD_legendary-sweep-5"

In [2]:
run = wandb.init()
links = {
    "GOOD_AVERAGE_glamorous-sweep-62": "mmil_vae_g2d/voice_distribution_and_genre_distribution_imbalance/model_epoch_100:v283",
    "GOOD_azure-sweep-54": "mmil_vae_g2d/voice_distribution_and_genre_distribution_imbalance/model_epoch_100:v279",
    "GOOD_apricot-sweep-17": "mmil_vae_g2d/voice_distribution_and_genre_distribution_imbalance/model_epoch_100:v242",
    "GOOD_hearty-sweep-60": "mmil_vae_g2d/voice_distribution_and_genre_distribution_imbalance/model_epoch_100:v280",
    "GOOD_worldly-sweep-22": "mmil_vae_g2d/voice_distribution_and_genre_distribution_imbalance/model_epoch_100:v245",
    "GOOD_legendary-sweep-5": "mmil_vae_g2d/voice_distribution_and_genre_distribution_imbalance/model_epoch_100:v230"
    
}
artifact = run.use_artifact(links[run_name], type='model')
artifact_dir = artifact.download()

In [3]:
import sys
sys.path.insert(0, "../..")

In [4]:
from helpers import load_variational_mgt_model
from model import GrooveTransformerEncoderVAE
import torch

Holoviews not installed, will not be able to generate violin plots


In [5]:
from helpers import load_variational_mgt_model   
import os

In [6]:
model = load_variational_mgt_model(os.path.join(artifact_dir, "100.pth"))

Offset activation is sigmoid, bias is initialized to 0.5


In [7]:
from data import load_gmd_hvo_sequences

train_set = load_gmd_hvo_sequences(
    dataset_setting_json_path = "../../data/dataset_json_settings/4_4_Beats_gmd.json", 
    subset_tag = "test", 
    force_regenerate=False)

INFO:data.Base.dataLoaders:Loading gmd dataset
INFO:data.Base.dataLoaders:Loading Cached Version from: data/gmd/resources/cached/beat_division_factor_[4]/drum_mapping_label_ROLAND_REDUCED_MAPPING/beat_type_['beat']_time_signature_['4-4']


In [8]:
gt_sample = train_set[0]
groove = torch.tensor([gt_sample.flatten_voices(reduce_dim=True)], dtype=torch.float32)
gt_sample.metadata

{'Source': 'Groove MIDI Dataset',
 'drummer': 'drummer1',
 'session': 'eval_session',
 'loop_id': 'drummer1/eval_session/10:000',
 'master_id': 'drummer1/eval_session/10',
 'style_primary': 'soul',
 'style_secondary': 'groove10',
 'bpm': '102',
 'beat_type': 'beat',
 'time_signature': '4-4',
 'full_midi_filename': 'drummer1/eval_session/10_soul-groove10_102_beat_4-4.mid',
 'full_audio_filename': 'drummer1/eval_session/10_soul-groove10_102_beat_4-4.wav'}

In [9]:
mu, logvar = model.encode_to_mu_logvar(groove)
latent_z = model.reparametrize(mu, logvar)

In [10]:
latent_z

tensor([[-9.7573e-01, -2.2011e+00, -2.6856e-01, -4.4952e-01, -6.5310e-01,
         -1.6098e+00,  1.1921e+00, -5.3905e-02,  3.0890e-01, -6.0284e-01,
         -2.7346e-01, -2.2806e-01,  4.7509e-01, -1.5193e+00,  1.8751e-01,
         -3.4495e-01, -1.9606e-01, -2.2361e+00, -1.1019e+00,  1.1106e+00,
         -7.1014e-01,  6.2116e-01, -4.5464e-01,  3.8978e-01,  8.6908e-01,
         -2.8577e+00,  3.0435e-01, -8.6802e-02,  1.2173e+00,  6.1665e-01,
          2.0839e-01,  2.4942e-02,  3.5906e-01,  5.6472e-01, -2.0695e+00,
         -6.7003e-01,  4.2757e-01, -2.5826e-01,  9.8807e-02,  1.2119e+00,
         -1.1405e-02,  9.3607e-01,  6.9518e-01, -9.1248e-01, -8.6465e-01,
         -1.7758e+00,  6.2461e-01,  2.2466e-01,  3.4439e-01, -5.2874e-01,
          1.3801e+00, -1.4026e+00, -8.2297e-01,  1.4343e+00, -2.0492e+00,
         -3.0631e-01,  4.3157e-01, -5.9786e-01,  1.9669e+00, -6.1456e-01,
          4.1027e-01,  1.0539e+00,  5.3148e-02, -5.7387e-01, -3.8489e-01,
         -9.7831e-01,  9.7513e-01, -7.

In [11]:
voice_thresholds = [0.5] * 9
voice_max_count_allowed = [32] * 9
h, v, o = model.sample(latent_z=latent_z,
                       voice_thresholds=voice_thresholds,
                       voice_max_count_allowed=voice_max_count_allowed,
                       return_concatenated=False,
                       sampling_mode=0)
print(h, v, o)

tensor([[[1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 1., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 1., 0., 0., 0., 0., 0., 0.],
         [0.,

In [12]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline


In [13]:
latents = []
labels = []
metadatas = []
use_all_styles = True

for gt_sample in train_set:
    if (use_all_styles or (gt_sample.metadata["style_primary"] in ["rock", "funk", "afrobeat"])):
        metadatas.append(gt_sample.metadata)
        labels.append(gt_sample.metadata["style_primary"])
        groove = torch.zeros((1, 32, 3))
        flattened_ = torch.tensor([gt_sample.flatten_voices(reduce_dim=True)], dtype=torch.float32)[:,:32, :]
        t_steps = flattened_.shape[1]
        groove[:, :t_steps, :] = flattened_
        mu, logvar = model.encode_to_mu_logvar(groove)
        latent_z = model.reparametrize(mu, logvar)
        latents.append(latent_z.detach().cpu().numpy())

latents = np.array(latents).squeeze(1)
features = np.expand_dims(latents, -1) # we use each dimension of latent_z as a feature
feature_labels = [f"z_{dim}" for dim in range(features.shape[1])]

In [14]:
import pandas as pd
data = {"style_primary": [style for style in labels]}
data.update({f"z_{dim_i}": latents[:, dim_i] for dim_i in range(features.shape[1])})
df = pd.DataFrame(data)
df.head()

,style_primary,z_0,z_1,z_2,z_3,z_4,z_5,z_6,z_7,z_8,...,z_246,z_247,z_248,z_249,z_250,z_251,z_252,z_253,z_254,z_255
0,soul,-1.492601,-1.215700,-1.691869,-0.048524,-0.300346,-0.306627,0.775103,1.167025,-0.622395,...,1.333052,1.577956,-0.744789,-0.303209,0.601953,0.227783,-0.399089,-0.444173,1.708916,-0.512316
1,soul,-1.276592,0.047340,0.608238,-1.150795,1.014565,-1.719121,0.444404,1.170950,0.562813,...,-0.497781,-0.976366,0.838607,0.215977,0.228641,0.894913,0.384004,-0.385644,-1.344402,0.111789
2,soul,-2.279988,0.456575,-0.099592,0.187153,0.962776,0.094378,0.633159,1.390931,2.363299,...,-1.082002,0.807119,1.122756,-0.312311,-0.287296,0.808156,-2.979047,0.189411,1.149416,2.046746
3,soul,-2.825154,-0.420306,1.056324,0.245516,0.700789,1.202725,-0.185915,1.130181,1.342632,...,1.738591,-1.813903,-0.106745,-0.371973,1.040680,0.709032,0.757395,0.845108,-0.242836,-0.249746
4,soul,-2.885637,-1.756703,0.354033,1.042696,0.293596,-0.253739,0.209462,0.647631,1.146277,...,-1.903604,0.054222,0.472195,-0.489930,0.576986,-0.856642,-0.473133,0.130949,0.352917,-0.664651


In [15]:
from umap import UMAP

In [16]:
embedding_dims = 2
umap = UMAP(n_components=embedding_dims, metric="euclidean", n_neighbors=20)
embeddings = umap.fit_transform(df.drop("style_primary", axis=1))

In [17]:
from bokeh.palettes import inferno, Category20b
from bokeh.core.enums import MarkerType

styles = sorted(df.style_primary.unique())
hues = inferno(len(styles))
hues = Category20b[20]
style_hue_map = {f"{style}": hues[i] for i, style in enumerate(styles)}

bases = [
    "circle",
    "diamond",
    "inverted_triangle",
    "square",
    "star",
    "triangle",
] * 4

marker_map = {style: bases[ix] for ix, style in enumerate(styles)}

In [18]:
from collections import OrderedDict
from bokeh.plotting import figure, show, save

embedding_dict = dict()
styles = df.style_primary


for ix, style in enumerate(styles):
    if(not style in embedding_dict):
        embedding_dict.update({style: ([], [])})
    embedding_dict[style][0].append(embeddings[ix][0])
    embedding_dict[style][1].append(embeddings[ix][1])
    

TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

p = figure(tools=TOOLS)

for key, values in embedding_dict.items():
    p.scatter(values[0], values[1],
              fill_color=style_hue_map[key], 
              line_color=None, legend_label=key, marker=marker_map[key], size=12)
p.legend.click_policy="hide"
save(p, f"UMAP_{run_name}.html")

'/home/dtic/Github/GrooveTransformer/eval/UMAP/UMAP_GOOD_legendary-sweep-5.html'

In [19]:
embedding_dict

{'soul': ([7.1174254,
   5.98397,
   9.1543,
   8.335508,
   9.406193,
   7.782061,
   8.458228,
   7.339868,
   8.7627125,
   9.198079,
   8.857986,
   9.349147,
   8.808778,
   8.423814,
   7.8564887,
   7.107931,
   7.927267,
   8.309752,
   7.034256,
   8.38329,
   8.276987,
   8.430526,
   8.215364,
   8.164844,
   8.221646,
   7.8899884,
   6.915372,
   6.128357,
   8.18717,
   8.380562,
   6.7004724,
   7.89981,
   7.369816,
   7.898284,
   8.358695,
   8.444289,
   8.314952,
   8.113494,
   8.688967,
   8.367962,
   7.486948,
   8.350775,
   7.812221,
   6.4039683,
   8.114319,
   7.4730854,
   7.8435903,
   7.9858828,
   6.6695967,
   9.526345,
   8.223806,
   7.0607843,
   7.35782,
   7.0582,
   6.7156734,
   6.918245,
   7.5209355,
   8.508323,
   6.353361,
   7.5048556,
   9.461937,
   7.8429065,
   7.2453704,
   6.6780753,
   7.660405,
   7.4983654,
   6.193707,
   6.0447335,
   6.9585056,
   7.37585,
   7.7117925,
   8.9462385,
   8.752583,
   7.8611803,
   6.8939977,
   

In [20]:
# from sklearn.manifold import TSNE
# umap = TSNE(n_components=embedding_dims)
# embeddings = umap.fit_transform(df.drop("style_primary", axis=1))

In [21]:
# import numpy as np

# from bokeh.plotting import figure, show, save
# from bokeh.io import output_notebook

# TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

# p = figure(tools=TOOLS)

# p.scatter(embeddings[:, 0], embeddings[:, 1],
         #  fill_color=colors, fill_alpha=0.6,
          # line_color=None)

# save(p, "TSNE.html") 

Exception in thread SystemMonitor:
Traceback (most recent call last):
  File "/home/dtic/miniconda3/envs/GrooveTransformer/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/home/dtic/miniconda3/envs/GrooveTransformer/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/home/dtic/miniconda3/envs/GrooveTransformer/lib/python3.9/site-packages/wandb/sdk/internal/system/system_monitor.py", line 118, in _start
    asset.start()
  File "/home/dtic/miniconda3/envs/GrooveTransformer/lib/python3.9/site-packages/wandb/sdk/internal/system/assets/cpu.py", line 166, in start
    self.metrics_monitor.start()
  File "/home/dtic/miniconda3/envs/GrooveTransformer/lib/python3.9/site-packages/wandb/sdk/internal/system/assets/interfaces.py", line 168, in start
    logger.info(f"Started {self._process.name}")
AttributeError: 'NoneType' object has no attribute 'name'
